# Creating a Cognite Function from a folder and scheduling it

In certain situations it will be convenient to separate the functionality of a task into multiple python modules or files. In this example, we create a function from a folder containing these modules. We also look at how we can schedule our Cognite Function to run at regular intervals. 

## Requirements
We will be using the [experimental Cognite SDK](https://cognite-sdk-experimental.readthedocs-hosted.com/en/latest/cognite.html#functions). 
To run this example, you first need to install this package:

```
pip install cognite-sdk-experimental
```

## Authentication

In order to connect to CDF we will authenticate using Azure AD. There are several ways to do this, all of them outlined in the [Authenticate with Azure AD](https://docs.cognite.com/dev/guides/sdk/python/python_auth_oidc/) documentation. We will in this example use [**client credentials**](https://docs.cognite.com/dev/guides/sdk/python/python_auth_oidc/#authenticate-with-client-secret). There are pros and cons to each authentication method. Here we choose client-credentials due to little additional work needed to get up and running.

This means that we need the following:

1. A `token_client_id` and a `token_client_secret`;
2. a `token_url` (can be inferred from the Azure tenant-ID); and finally,
3. a list of `token_scopes` (can usually be inferred from the CDF cluster). 

We start by importing the experimental `CogniteClient` and log in by passing in the required credentials. We use `getpass` to avoid having to paste confidential information directly in the noteobok cell. 

In [ ]:
from getpass import getpass
from cognite.experimental import CogniteClient

project = "<CDF project>"  # Fill in your project here
cdf_cluster = "<CDF cluster>"  # Fill in the cluster your project is running in (for instance 'api'/'westeurope-1')

tenant_id = "<Tenant ID>"  # Fill in your Azure AD tenant ID here
token_client_id = "<Token Client ID>"  # Fill in your Client ID here
token_client_secret = getpass("Paste your `token_client_secret` in here: ")

token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
token_scopes = [f"https://{cdf_cluster}.cognitedata.com/.default"]

client = CogniteClient(
    project=project,
    server=cdf_cluster,
    client_name="my second Cognite Function",
    token_url=token_url,
    token_scopes=token_scopes,
    token_client_id=token_client_id,
    token_client_secret=token_client_secret,
)

## Defining our function
In this example we define a function in two python modules, a [handler.py](handler.py) which contains the main `handle` function and a [helper.py](helper.py) module. The latter module is called by the former to show how module dependencies can be used. The helper module is imported in the handler by writing `import helper` which we can do because it's contained in the same folder as the [handler.py](handler.py) module. 

Please note that:
* The `handle` function in the file [handler.py](handler.py) must always be specified, since this is the entry point of the deployed function. The file `handler.py` must always be present in the root folder, unless otherwise specified via the optional `function_path` argument to `client.functions.create()`.
* The `handle` function must have as arguments a subset of `(client, data, secrets, function_call_info)`. 

In this example we will only need the `data` argument. The data we give when calling the function, is passed through this argument.

## Testing the function locally
Before we actually deploy the function to CDF, we can test it locally. First take a look at the two files [handler.py](handler.py) and [helper.py](helper.py), before you test the code.

In [ ]:
import handler

handler.handle(data={"value": 2.0})

## Deploying the function to CDF
Next we deploy our function to CDF, by setting the argument `folder` to the relevant folder. At least one of the files in the folder must be named `handler.py` (unless otherwise specified, as explained in the previous section), and this file must have a function named `handle`, which is the function that will be executed in CDF.

First let's create a unique string as a function external id. 

In [ ]:
fn_name = "my-transformer-function"
external_id = "my-transformer-function"

We now create the function. We also optionally add a description and an owner. 

In [ ]:
function = client.functions.create(
    name=fn_name,
    external_id=external_id,
    folder=".",  # We will upload this directory
    description="Square an input value",
    owner="Ola Normann",
)

Next we can retrieve the function to see its status. It will start out as `Queued` and then go to `Deploying`. After a couple of minutes, the status will be `Ready`, and you can start to call your function. **NB:** All calls to the function before the function is `Ready` will fail.

In [ ]:
# Repeat until status is ready
while function.status != "Ready":
    function.update()

    if function.status == "Failed":
        print("Failed to deploy function")
        break
else:
    print("Function is successfully deployed")

When the function is `Ready`, we can call it directly on the function object.

In [ ]:
call = function.call(data={"value": 2})
call

Note that the above calls are to the function in CDF, not the local function.

To retrieve the response from the function, simply invoke the `get_response()` method on the call object:

In [ ]:
response = call.get_response()
response

## Handling failures
If you get a failure you can look at the logs

In [ ]:
call_bad = function.call(data={"foo": 2})
call_bad

In [ ]:
call_bad.get_logs()

In [ ]:
call_bad.get_logs()[-2:]  # Here we see the error

### Schedules

Schedules are objects that make the functions run at pre-defined intervals. For example, we can run a function every Monday at 7am or every other friday, etc. A schedule is defined with an associated function id and some input data. The interval syntax we use is called unix cron syntax, which is a string that contains the necessary information to decode a wide variety of running intervals. More information about cron syntax can be found here: https://crontab.guru.

Below is an example of a schedule that runs every minute on our defined function. Note that we can define multiple schedules on each function, each one with different input data.

In order to define a schedule we need to also pass in a set of client credentials. In this example, we will for simplicity use the client credentials we used when instantiating the client. However, in production settings you should use a dedicated set of credentials. 

In [ ]:
client_credentials = {
    "client_id": token_client_id,
    "client_secret": token_client_secret,
}
schedule = client.functions.schedules.create(
    name="run-function-every-minute",
    cron_expression="* * * * *",  # the cron expression runs every minutes
    function_id=function.id,  # we specify the ID of the function we want to schedule
    client_credentials=client_credentials,  # this is a dictionary with 'client_secret' and 'client_id'
    data={"value": 2},  # this is the data we wish to call the function with
    description="This schedule runs the function every minute",
)

## List calls
We can list all calls that have been made to the function:

In [ ]:
calls = function.list_calls()
calls

In [ ]:
# Get the response of the last function call
calls[-1].get_response()

## Cleanup
Finally, we clean after ourselves by deleting our function and associated schedule.

In [ ]:
client.functions.delete(id=function.id)